In [131]:
import requests
from bs4 import BeautifulSoup
import csv

class Scraper ():

    
    def __init__(self):
        self.url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&tg=0&date="
        
        
    def get_html(self, date):
        res = requests.get(self.url+str(date))

        if res.status_code != 200:
            print("Request Error : ", res.status_code)
    
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        return soup

 
    def change_direction(L):
        if L == 'na':
            L = '='
        elif L == 'up':
            L = '+'
        else:
            L = '-'

        return L

 
    
    def get_cards(self, soup, date):
        #job카드를 가져오자
        listRanking = soup.find(class_ = "list_ranking")
       
        movieName = []
        movieUpdown = []
        movieNum = []

        for tr_tag in listRanking.find_all('tr'):
            tag = tr_tag.find(class_='tit3')
            if tag:
                movieName.append(tag.text.replace("\n",""))

            updown = tr_tag.find(class_='arrow')
            if updown:
                movieUpdown.append(change_direction(updown["alt"]))

            num = tr_tag.find(class_='range ac')
            if num:
                movieNum.append(num.text)

            
        self.write_csv(date, movieName, movieUpdown, movieNum)
        
        
        
    def write_csv(self, date, movieName, movieUpdown, movieNum):
        #파일은 데이터 추가, newline 시 공백 추가
        file = open("movieRank.csv", "a", newline="", encoding  = 'euc-kr')
        
        wr = csv.writer(file)
        for i in range(len(movieName)):
            wr.writerow([str(date), str(i + 1), movieName[i], movieUpdown[i], movieNum[i]])
            
        file.close()
        
    def scrap(self, startdate, finishdate):
        
        #처음에 실행할 때는 w로 실행하여, 기존의 데이터가 있으면 초기화
        file = open("movieRank.csv", "w", newline = "", encoding  = 'euc-kr')
        wr = csv.writer(file)
        wr.writerow(["Date","Rank.", "Name", "UpDown", "Change_Amount"])
        file.close()
        
        for i in range(finishdate - startdate +1):
            soup = self.get_html(startdate + i)
            self.get_cards(soup, startdate + i)

In [132]:
s = Scraper()
s.scrap(20200413,20200414)